In [1]:
import pandas as pd
import string
import nltk
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, precision_recall_curve, accuracy_score
from sklearn.metrics import roc_auc_score
# !pip install imblearn --quiet
# !pip install llm2vec --quiet
# !pip install flash-attn --no-build-isolation
# import torch
# from llm2vec import LLM2Vec
import pickle


import warnings

warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data preprocessing

In [3]:
test_data = pd.read_csv('/content/drive/MyDrive/Agent_LLM_paper/drugsComTest_raw.tsv',sep = '\t')
train_data = pd.read_csv('/content/drive/MyDrive/Agent_LLM_paper/drugsComTrain_raw.tsv',sep = '\t')
test_data['label'] = test_data['rating'].apply(lambda x: 0 if x <= 5 else 1)
train_data['label'] = train_data['rating'].apply(lambda x: 0 if x <= 5 else 1)

In [4]:
y_train, y_test = train_data['label'], test_data['label']

In [5]:
with open('/content/drive/MyDrive/Agent_LLM_paper/x_train_llm2vec.pkl', 'rb') as fichier:
    x_train = pickle.load(fichier)

with open('/content/drive/MyDrive/Agent_LLM_paper/x_test_llm2vec.pkl', 'rb') as fichier:
    x_test = pickle.load(fichier)

# 1. RandomForestClassifier

In [6]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0)

In [7]:
model.fit(x_train, y_train)

RandomForestClassifier(random_state=0)

In [8]:
model.score(x_train, y_train)

0.9999132035933712

In [9]:
y_pred = model.predict(x_test)

In [10]:

target_names = ['negative', 'positive']
y_predictions = (y_pred > 0.5).astype(int)
predictions = pd.DataFrame(y_predictions, columns=['Prediction'])
predictions.reset_index(inplace=True)
predictions.columns = ['Index', 'Prediction']
auc = roc_auc_score(y_test, y_pred)
print(f"L'AUC est de : {auc}")
print(classification_report(y_test, predictions['Prediction'], target_names=target_names))

L'AUC est de : 0.9031749529934363
              precision    recall  f1-score   support

    negative       0.94      0.83      0.88     16207
    positive       0.93      0.98      0.95     37559

    accuracy                           0.93     53766
   macro avg       0.94      0.90      0.92     53766
weighted avg       0.93      0.93      0.93     53766



In [11]:
x_total = np.concatenate((x_train, x_test), axis=0)

In [12]:
y_pred_glove_lstm = model.predict(x_total)

In [13]:
import pickle
with open('/content/drive/MyDrive/Agent_LLM_paper/y_pred_llm2vec_rand_forest.pkl', 'wb') as fichier:
    pickle.dump(y_pred_glove_lstm, fichier)

# 2. ExtraTreesClassifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model4 = ExtraTreesClassifier(random_state=0)

In [ ]:
model4.fit(x_train, y_train)

ExtraTreesClassifier(random_state=0)

In [ ]:
model4.score(x_train, y_train)

0.9999132035933712

In [ ]:
y_pred = model4.predict(x_test)

In [ ]:
target_names = ['negative', 'positive']
y_predictions = (y_pred > 0.5).astype(int)
predictions = pd.DataFrame(y_predictions, columns=['Prediction'])
predictions.reset_index(inplace=True)
predictions.columns = ['Index', 'Prediction']
auc = roc_auc_score(y_test, y_pred)
print(f"L'AUC est de : {auc}")
print(classification_report(y_test, predictions['Prediction'], target_names=target_names))

L'AUC est de : 0.891573975646778
              precision    recall  f1-score   support

    negative       0.95      0.80      0.87     16207
    positive       0.92      0.98      0.95     37559

    accuracy                           0.93     53766
   macro avg       0.94      0.89      0.91     53766
weighted avg       0.93      0.93      0.93     53766



# 3. Bi-LSTM

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout

embedding_dim = 4096
num_classes = 2

In [ ]:
x_train2 = np.expand_dims(x_train, axis=1)
x_test2 = np.expand_dims(x_test, axis=1)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150, return_sequences=True),input_shape=(1, embedding_dim)))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Dense(80, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x_train2, y_train, epochs=5)

Epoch 1/5
5041/5041 [==============================] - 95s 18ms/step - loss: 0.3915 - accuracy: 0.8298
Epoch 2/5
5041/5041 [==============================] - 89s 18ms/step - loss: 0.3758 - accuracy: 0.8382
Epoch 3/5
5041/5041 [==============================] - 90s 18ms/step - loss: 0.3685 - accuracy: 0.8413
Epoch 4/5
5041/5041 [==============================] - 89s 18ms/step - loss: 0.3650 - accuracy: 0.8411
Epoch 5/5
5041/5041 [==============================] - 88s 17ms/step - loss: 0.3619 - accuracy: 0.8447


In [ ]:
y_pred = model.predict(x_test2)

1681/1681 [==============================] - 7s 4ms/step


In [ ]:
target_names = ['negative', 'positive']
y_predictions = (y_pred > 0.5).astype(int)
predictions = pd.DataFrame(y_predictions, columns=['Prediction'])
predictions.reset_index(inplace=True)
predictions.columns = ['Index', 'Prediction']
auc = roc_auc_score(y_test, y_pred)
print(f"L'AUC est de : {auc}")
print(classification_report(y_test, predictions['Prediction'], target_names=target_names))

L'AUC est de : 0.9070213083789327
              precision    recall  f1-score   support

    negative       0.72      0.78      0.75     16207
    positive       0.90      0.87      0.89     37559

    accuracy                           0.84     53766
   macro avg       0.81      0.82      0.82     53766
weighted avg       0.85      0.84      0.84     53766



# 4. DNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


input_dim = 4096
output_dim = 2

model = Sequential()


model.add(Dense(256, activation='relu', input_shape=(input_dim,)))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
5041/5041 [==============================] - 28s 5ms/step - loss: 0.3907 - accuracy: 0.8322
Epoch 2/5
5041/5041 [==============================] - 28s 5ms/step - loss: 0.3396 - accuracy: 0.8504
Epoch 3/5
5041/5041 [==============================] - 27s 5ms/step - loss: 0.3297 - accuracy: 0.8547
Epoch 4/5
5041/5041 [==============================] - 27s 5ms/step - loss: 0.3230 - accuracy: 0.8578
Epoch 5/5
5041/5041 [==============================] - 27s 5ms/step - loss: 0.3188 - accuracy: 0.8597


In [ ]:
y_pred = model.predict(x_test)

1681/1681 [==============================] - 3s 2ms/step


In [ ]:
target_names = ['negative', 'positive']
y_predictions = (y_pred > 0.5).astype(int)
predictions = pd.DataFrame(y_predictions, columns=['Prediction'])
predictions.reset_index(inplace=True)
predictions.columns = ['Index', 'Prediction']
auc = roc_auc_score(y_test, y_pred)
print(f"L'AUC est de : {auc}")
print(classification_report(y_test, predictions['Prediction'], target_names=target_names))

L'AUC est de : 0.928207753323989
              precision    recall  f1-score   support

    negative       0.76      0.80      0.78     16207
    positive       0.91      0.89      0.90     37559

    accuracy                           0.86     53766
   macro avg       0.83      0.84      0.84     53766
weighted avg       0.86      0.86      0.86     53766



# 5. CNN

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(embedding_dim,1)))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(80, activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
x_train_reshaped = x_train.reshape(-1, embedding_dim, 1)
x_test2 = x_test.reshape(-1, embedding_dim, 1)

In [ ]:
model.fit(x_train_reshaped, y_train, epochs=5)

Epoch 1/5
5041/5041 [==============================] - 1179s 234ms/step - loss: 0.3728 - accuracy: 0.8377
Epoch 2/5
5041/5041 [==============================] - 1156s 229ms/step - loss: 0.3369 - accuracy: 0.8526
Epoch 3/5
5041/5041 [==============================] - 1159s 230ms/step - loss: 0.3250 - accuracy: 0.8583
Epoch 4/5
5041/5041 [==============================] - 1151s 228ms/step - loss: 0.3146 - accuracy: 0.8629
Epoch 5/5
5041/5041 [==============================] - 1157s 229ms/step - loss: 0.3060 - accuracy: 0.8676


In [ ]:
y_pred = model.predict(x_test2)

1681/1681 [==============================] - 91s 54ms/step


In [ ]:
target_names = ['negative', 'positive']
y_predictions = (y_pred > 0.5).astype(int)
predictions = pd.DataFrame(y_predictions, columns=['Prediction'])
predictions.reset_index(inplace=True)
predictions.columns = ['Index', 'Prediction']
auc = roc_auc_score(y_test, y_pred)
print(f"L'AUC est de : {auc}")
print(classification_report(y_test, predictions['Prediction'], target_names=target_names))

L'AUC est de : 0.9261545701815808
              precision    recall  f1-score   support

    negative       0.80      0.73      0.76     16207
    positive       0.89      0.92      0.90     37559

    accuracy                           0.86     53766
   macro avg       0.84      0.83      0.83     53766
weighted avg       0.86      0.86      0.86     53766

